In [ ]:
import pandas as pd
import heapq
from math import radians, cos, sin, sqrt, atan2

file_path = 'C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/path_algorithm/dataset/liste-des-gares.csv'

def load_graph_from_ratp_csv(file_path):
    ratp_data = pd.read_csv(file_path, delimiter=';')
    graph = {}

    for _, row in ratp_data.iterrows():
        source = row['LIBELLE']
        source_coords = (row['Y_WGS84'], row['X_WGS84'])
        
        graph[source] = {'coords': source_coords, 'connections': []}
        
        connections = ratp_data[ratp_data['CODE_LIGNE'] == row['CODE_LIGNE']]
        
        for _, conn_row in connections.iterrows():
            destination = conn_row['LIBELLE']
            destination_coords = (conn_row['Y_WGS84'], conn_row['X_WGS84'])
            
            if source != destination:
                weight = haversine(source_coords, destination_coords)
                graph[source]['connections'].append((destination, weight))
                if destination not in graph:
                    graph[destination] = {'coords': destination_coords, 'connections': []}
                graph[destination]['connections'].append((source, weight))
    
    return graph

def a_star_path(graph, start, end):
    if start not in graph:
        return float('inf'), []
    if end not in graph:
        return float('inf'), []
    
    def heuristic(node, end):
        return haversine(graph[node]['coords'], graph[end]['coords']) if node in graph else float('inf')
    
    priority_queue = [(0 + heuristic(start, end), 0, start, [])]
    visited = set()
    g_scores = {start: 0}
    
    while priority_queue:
        _, current_g, current_node, path = heapq.heappop(priority_queue)
        
        if current_node in visited:
            continue
        visited.add(current_node)
        
        path = path + [current_node]
        
        if current_node == end:
            return current_g, path
        
        for neighbor, weight in graph[current_node]['connections']:
            if neighbor not in visited:
                g_score = current_g + weight
                f_score = g_score + heuristic(neighbor, end)  # f(n) = g(n) + h(n)
                
                if neighbor not in g_scores or g_score < g_scores[neighbor]:
                    g_scores[neighbor] = g_score
                    heapq.heappush(priority_queue, (f_score, g_score, neighbor, path))
    
    return float('inf'), []  # If no path exists

def haversine(coord1, coord2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

graph = load_graph_from_ratp_csv(file_path)

start_station = "Ermont-Eaubonne"
end_station = "La Défense"
distance, path = a_star_path(graph, start_station, end_station)

print(f"Shortest path from {start_station} to {end_station}: {path} with distance {distance:.2f} km")

Shortest path from Ermont-Eaubonne to La Défense: ['Ermont-Eaubonne', 'Neuilly-Porte-Maillot', 'Champ-de-Mars-Tour-Eiffel', 'Viroflay-Rive-Gauche', 'Trappes', 'Versailles-Chantiers', 'Noisy-le-Roi', 'St-Nom-la-Bretêche-Forêt-de-Marly', 'St-Cloud', 'Clichy-Levallois', 'La Défense'] with distance 81.88 km
